# **Example: Calculating Underground Fluxes**

This file demonstrates how to use MUTE to calculate underground fluxes for a lab located 3.7 km.w.e. under rock.

## Import Packages

In [1]:
import numpy as np

import mute.constants as mtc
import mute.surface as mts
import mute.propagation as mtp
import mute.underground as mtu

*************************************************************************
*                                                                       *
*                ███████████████████████████████████████                *
*                ▓  ▓▓▓▓  ▓▓  ▓▓▓▓  ▓▓        ▓▓       ▓                *
*                ▓   ▓▓   ▓▓  ▓▓▓▓  ▓▓▓▓▓  ▓▓▓▓▓  ▓▓▓▓▓▓                *
*                ▒        ▒▒  ▒▒▒▒  ▒▒▒▒▒  ▒▒▒▒▒       ▒                *
*                ▒  ▒  ▒  ▒▒  ▒▒▒▒  ▒▒▒▒▒  ▒▒▒▒▒  ▒▒▒▒▒▒                *
*                ░  ░░░░  ░░░░    ░░░░░░░  ░░░░░       ░                *
*                ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                *
*                   https://github.com/wjwoodley/mute                   *
*                                                                       *
* Author:  William Woodley                                              *
* Version: 3.0.0                                                        *
*                                     

## Set the Constants

In [2]:
mtc.set_verbose(2)
mtc.set_output(True)
mtc.set_lab("Example")
mtc.set_overburden("flat")
mtc.set_vertical_depth(3.7)
mtc.set_medium("rock")
mtc.set_reference_density(2.65)
mtc.set_n_muon(1000000)

## Check the Slant Depths

Check the number and value of the slant depths that the underground fluxes will be calculated with. Because the vertical depth was set to ``3.7`` above, the slant depths should start at 3.7 km.w.e. The number of slant depths should be reduced from the default 28 to 22.

In [3]:
print(mtc.slant_depths)
print(len(mtc.slant_depths))

[ 3.7  4.   4.5  5.   5.5  6.   6.5  7.   7.5  8.   8.5  9.   9.5 10.
 10.5 11.  11.5 12.  12.5 13.  13.5 14. ]
22


The underground fluxes will be calculated for the zenith angles corresponding to these slant depths. The correspondence is given by this equation:

$$\theta=\arccos\left(\frac{h}{X}\right)=\arccos\left(\frac{3.7\ \mathrm{km.w.e.}}{X}\right)$$

## Calculate the Underground Fluxes

The ``mtu.calc_u_fluxes()`` function will return a matrix of shape ``(91, 22)`` for the 91 energies in the grid given by ``mtc.ENERGIES``, and the 22 angles corresponding to the slant depths above.

In [4]:
u_fluxes = mtu.calc_u_fluxes(model = "daemonflux")

print(u_fluxes)
print(u_fluxes.shape)

Calculating underground fluxes.
Loading surface fluxes for daemonflux.
Loaded surface fluxes.
Loading survival probabilities from data/survival_probabilities/rock_2.65_1000000_survival_probabilities.npy.
Loaded survival probabilities.
Finished calculating underground fluxes.
Underground fluxes written to data/underground/Example_underground_fluxes.txt.
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.16703202e-11 2.87100319e-12 7.85940435e-13 ... 2.15525490e-17
  1.40345948e-17 1.03211958e-17]
 [1.90158930e-10 7.66299041e-12 2.48824130e-12 ... 9.69024934e-17
  5.98686426e-17 2.71763838e-17]
 ...
 [1.76953790e-30 1.01423372e-30 5.28797216e-31 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.06786863e-31 3.64688148e-31 8.43004699e-32 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.95098752e-31 2.78689621e-34 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(91, 22)


If the full tensor is needed, this can be obtained by setting ``full_tensor`` to ``True`` in the function call. This will return a three-dimensional array of shape ``(28, 91, 20)``, for the default slant depths, energies, and surface flux zenith angles given by ``mtc._SLANT_DEPTHS``, ``mtc.ENERGIES``, and ``mtc.ANGLES_FOR_S_FLUXES`` respectively. The output file will also contain the full tensor.

In [5]:
u_fluxes_full = mtu.calc_u_fluxes(full_tensor = True, model = "daemonflux")

print(u_fluxes_full[0, :5, :5])
print(u_fluxes_full.shape)

Calculating underground fluxes.
Loading surface fluxes for daemonflux.
Loaded surface fluxes.
Survival probabilities already loaded for rock with density 2.65 gcm^-3 and 1000000 muons.
Finished calculating underground fluxes.
Underground fluxes written to data/underground/Example_underground_fluxes.txt.
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [6.16703202e-11 6.17509618e-11 6.19923356e-11 6.23927929e-11
  6.29516409e-11]
 [1.90158930e-10 1.90406375e-10 1.91147018e-10 1.92375801e-10
  1.94090450e-10]
 [2.57498340e-10 2.57834063e-10 2.58838938e-10 2.60506102e-10
  2.62832552e-10]
 [2.81240649e-10 2.81606777e-10 2.82702660e-10 2.84520812e-10
  2.87057890e-10]]
(28, 91, 20)


## Calculating Fluxes from Pre-Calculated Matrices

If the code already has a surface flux matrix and / or survival probability tensor defined, they can be passed directly into the ``mtu.calc_u_fluxes()`` function. This is especially useful when dealing with multiple surface flux matrices and / or survival probability tensors in the same file, like when calculating various quantities by looping over variables. For example, to calculate both the surface fluxes and underground fluxes for different locations, the following can be done:

In [6]:
locations = ["SoudanMine", "SanGrasso", "Tokyo"]

all_s_fluxes = []
all_u_fluxes = []

sp_tensor = mtp.load_survival_probability_tensor_from_file()

for loc in locations:
    
    s_fluxes = mts.calc_s_fluxes(atmosphere = "msis00", location = loc, month = "January")
    u_fluxes = mtu.calc_u_fluxes(s_fluxes = s_fluxes, survival_probability_tensor = sp_tensor)
    
    all_s_fluxes.append(s_fluxes)
    all_u_fluxes.append(u_fluxes)

Survival probabilities already loaded for rock with density 2.65 gcm^-3 and 1000000 muons.
Calculating surface fluxes for SoudanMine using gsf and sibyll23c.
MCEqRun::set_interaction_model(): SIBYLL23C


.local/lib/python3.12/site-packages/crflux/models.py:1068: DeprecationWarning: Please import `InterpolatedUnivariateSpline` from the `scipy.interpolate` namespace; the `scipy.interpolate.fitpack2` namespace is deprecated and will be removed in SciPy 2.0.0.
  self.p_frac_spl, self.p_flux_spl, self.n_flux_spl = pickle.load(


MCEqRun::set_density_model(): Setting density profile to CORSIKA ('BK_USStd', None)
MCEqRun::set_primary_model(): GlobalSplineFitBeta 
MCEqRun::set_density_model(): Setting density profile to MSIS00 ('SoudanMine', 'January')


100%|██████████| 20/20 [01:03<00:00,  3.17s/it]


Finished calculating surface fluxes.
Surface fluxes written to data/surface/surface_fluxes_SoudanMine_January_sibyll23c_gsf.txt.
Calculating underground fluxes.
Finished calculating underground fluxes.
Underground fluxes written to data/underground/Example_underground_fluxes.txt.
Calculating surface fluxes for SanGrasso using gsf and sibyll23c.
MCEqRun::set_interaction_model(): SIBYLL23C


...

Finished calculating surface fluxes.
Surface fluxes written to data/surface/surface_fluxes_SanGrasso_January_sibyll23c_gsf.txt.
Calculating underground fluxes.
Finished calculating underground fluxes.
Underground fluxes written to data/underground/Example_underground_fluxes.txt.
Calculating surface fluxes for Tokyo using gsf and sibyll23c.
MCEqRun::set_interaction_model(): SIBYLL23C


.local/lib/python3.12/site-packages/crflux/models.py:1068: DeprecationWarning: Please import `InterpolatedUnivariateSpline` from the `scipy.interpolate` namespace; the `scipy.interpolate.fitpack2` namespace is deprecated and will be removed in SciPy 2.0.0.
  self.p_frac_spl, self.p_flux_spl, self.n_flux_spl = pickle.load(


MCEqRun::set_density_model(): Setting density profile to CORSIKA ('BK_USStd', None)
MCEqRun::set_primary_model(): GlobalSplineFitBeta 
MCEqRun::set_density_model(): Setting density profile to MSIS00 ('Tokyo', 'January')


100%|██████████| 20/20 [01:03<00:00,  3.18s/it]

Finished calculating surface fluxes.
Surface fluxes written to data/surface/surface_fluxes_Tokyo_January_sibyll23c_gsf.txt.
Calculating underground fluxes.
Finished calculating underground fluxes.
Underground fluxes written to data/underground/Example_underground_fluxes.txt.


Like this, the loop will not have to be broken to calculate the underground fluxes, and the surface fluxes will not have to be calculated again inside the ``mtu.calc_u_fluxes()`` function like they are in the ``mts.calc_s_fluxes()`` function, because they are being passed into it directly.

The functions for surface intensities and total fluxes can take ``s_fluxes`` matrices in as well, and the functions for underground intensities and total fluxes can take in both ``s_fluxes`` and ``survival_probability_tensor`` arguments.